<a href="https://colab.research.google.com/github/kaydee24/Applied-Machine-Learning/blob/master/Assignment_3_Kamal_Dhull.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
##This block is only for access of files using google drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#For accessing any file from google drive, first share it for public access. Copy its id from last part of its address. Then specify the two lines below.
downloaded = drive.CreateFile({'id':"NDg5NjM5Nzc2MjRa/a/NTAwMzkzNjk0NTRa"})   # replace the id with id of file you want to access
downloaded.GetContentFile('spam.csv')        # replace the file name with your file

In [0]:
import pandas as pd
import numpy as np
import string

#import the data file
filename = 'spam.csv' 

df_sms = pd.read_csv('spam.csv',encoding='latin-1')
df_sms.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [0]:
#Remove the unwanted columns
df_sms = df_sms.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)
df_sms = df_sms.rename(columns={"v1":"label", "v2":"sms"})
df_sms.head()

,label,sms
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [0]:
#Print number of records
L = len(df_sms)
print(L)
#Example of accessing a column in pandas dataframe
df_sms.sms

5572


0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                Will Ì_ b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: sms, Length: 5572, dtype: object

In [0]:
#Define a Function to convert sms text to Lower case and remove stop words, punctuation and numbers

def preprocess_Text(input_Text):
  input_Text = input_Text.lower();
  stopwords = ['the','what','is','a','an','of', 'that']
  querywords = input_Text.split()

  resultwords  = [word for word in querywords if word not in stopwords]
  result = ' '.join(resultwords)

  exclude = set(string.punctuation)
  result = ''.join(ch for ch in result if ch not in exclude)

  exclude = set('0123456789')
  result = ''.join(ch for ch in result if ch not in exclude)

  return result;

In [0]:
#Preprocess all the sms texts
L = len(df_sms)
for i in range(0,L-1):
  df_sms['sms'][i] = preprocess_Text(df_sms['sms'][i])

In [0]:
#Divide the dataframes into training and and testing set
from sklearn.utils import shuffle
df_sms = shuffle(df_sms)
training_Subset = df_sms.iloc[:round(len(df_sms)*0.9),:]   #90% data into training
test_Subset = df_sms.iloc[round(len(df_sms)*0.9):,:]       #10% data into testing
spam_Subset = training_Subset.query('label == "spam"')
ham_Subset = training_Subset.query('label == "ham"');

In [0]:
#combine all text into one large paragraph which shall be used to list unique words
L = len(training_Subset);
all_Text = ""
for i in training_Subset.index:
  all_Text = all_Text + " "+training_Subset['sms'][i];

In [0]:
#make a table with all unique words
allWords = all_Text.split()

row_Names = [] 
for i in allWords:
      if not i in row_Names:
          row_Names.append(i);
print(row_Names)

['k', 'pa', 'had', 'your', 'lunch', 'aha', 'hi', 'its', 'kate', 'how', 'evening', 'i', 'hope', 'can', 'see', 'you', 'tomorrow', 'for', 'bit', 'but', 'have', 'to', 'bloody', 'babyjontet', 'txt', 'back', 'if', 'u', 'xxx', 'anything', 'lor', 'no', 'problem', 'talk', 'later', 'where', 'wuld', 'be', 'without', 'my', 'baby', 'thought', 'alone', 'mite', 'break', 'me', 'and', 'donåõt', 'wanna', 'go', 'crazy', 'everyboy', 'needs', 'his', 'lady', 'xxxxxxxx', 'ok', 'thanks', 've', 'lovely', 'wisheds', 'rock', 'yup', 'next', 'stop', 'urgent', 'this', 'nd', 'attempt', 'contact', 'uu', 'won', 'å£call', 'b', 'tcsbcmwcnxxcallcostppmmobilesvary', 'maxå£', 'hey', 'boys', 'want', 'hot', 'pics', 'sent', 'direct', 'ur', 'phone', 'porn', 'hrs', 'free', 'then', 'just', 'p', 'per', 'day', 'text', 'stopbcm', 'sf', 'wcnxx', 'kkapo', 'kgood', 'movie', 'heygreat', 'dealfarm', 'tour', 'am', 'pm', 'pax', 'deposit', 'by', 'may', 'wonders', 'in', 'world', 'th', 'style', 'smile', 'personality', 'rd', 'nature', 'sms', 

In [0]:
#For each word find inspam probability and in-ham probability
word = '';
inSpamCount = 0;
inHamCount = 0;
columns = ['inSpamProbability','inHamProbability']
probability_Table = pd.DataFrame(index=row_Names, columns=columns)
for word in row_Names:
  inSpamCount = 0;
  inHamCount = 0;
  for i in spam_Subset['sms']:
    if(i.find(word)==0):
      inSpamCount = inSpamCount+1;
     
      
  for i in ham_Subset['sms']:
    if(i.find(word)==0):
       inHamCount = inHamCount+1;
      
      
  probability_Table.at[word, 'inSpamProbability'] = inSpamCount/len(spam_Subset);
  probability_Table.at[word,'inHamProbability'] = inHamCount/len(ham_Subset);


In [0]:
probability_Table.sort_values("inSpamProbability", axis = 0, ascending = False, 
                 inplace = True, na_position ='first') 

In [0]:
probability_Table

,inSpamProbability,inHamProbability
u,0.121212,0.0192882
f,0.1,0.013318
ur,0.0893939,0.00137773
y,0.0863636,0.0684271
you,0.0848485,0.0220436
...,...,...
sura,0,0
kz,0,0
wit,0,0.000229621
skyped,0,0


In [0]:
#drop rows wherever spam or ham is zero probability
probability_Table = probability_Table[(probability_Table[['inSpamProbability','inHamProbability']] != 0).all(axis=1)]

In [0]:
#Question_1:
def check_message(test_msg):
 test_msg = test_msg.split()
 word=""
 spam_prob=1
 ham_prob=1
 for word in test_msg:
  for words in probability_Table.index:
   if(word == words):
    spam_prob = spam_prob * probability_Table['inSpamProbability'][word]
    ham_prob = ham_prob * probability_Table['inHamProbability'][word]

 if(spam_prob >= ham_prob):
   return "spam"
 else:
   return "ham"




In [0]:
test_label = []
for test_inp in test_Subset['sms']:
 out_label = check_message(test_inp)
 test_label.append(out_label)
test_Subset['prediction'] = test_label

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [0]:
test_Subset

,label,sms,predicted_label,prediction
5213,ham,pa but not selected,ham,ham
3701,ham,hey loverboy i love you i had to tell i look...,ham,ham
5282,ham,i have gone into get info bt dont know to do,spam,spam
4766,spam,your unique user id for removal send stop to ...,spam,spam
4108,ham,i want to lick your pussy now,spam,spam
...,...,...,...,...
3402,ham,good night my dear sleepwellamptake care,spam,spam
1217,ham,damn can you make it tonight or do you want to...,spam,spam
2060,ham,i did one slice and one breadstick lol,ham,ham
5377,spam,current leading bid to pause this auction sen...,spam,spam


In [0]:
L = len(test_Subset)
test_spam = len(test_Subset.query('label == "spam"'))
test_spam_data = test_Subset.query('label == "spam"')
test_ham = len(test_Subset.query('label == "ham"'))
test_ham_data = test_Subset.query('label == "ham"')

print(test_spam)
print(test_ham)


87
470


In [0]:
#Question_2:
#True Negative Rate
ptn = len(test_spam_data.query('prediction == "spam"'))
#TN Rate
TN = ptn / test_spam
print(TN)

0.9310344827586207


In [0]:
#True Positive Rate
ptp = len(test_ham_data.query('prediction == "ham"'))

#TP Rate
TP = ptp / test_ham
print(TP)

0.451063829787234


In [0]:
#False Negative Rate
pfn = len(test_spam_data.query('prediction == "ham"'))

#FN Rate
FN = pfn / test_spam
print(FN)

0.06896551724137931


In [0]:
#False Positive Rate
pfp = len(test_ham_data.query('prediction == "spam"'))

#FP Rate
FP = pfp / test_ham
print(FP)

0.548936170212766


In [0]:
Accuracy = (TP + TN) / (TP + TN + FP + FN)
print(Accuracy)

0.6910491562729274


In [0]:
Error_rate = (FP + FN) / (TP + TN + FP + FN)
print(Error_rate)

0.3089508437270727
